# Combine CSV files with name rules

In [4]:
import pandas as pd
dfs = []

for cal_year in range(2021,2025):
    df = pd.read_csv(f'result/jp225_from_1-{cal_year}_to_12-{cal_year}.csv', header=None)
    dfs.append(df)
    print(cal_year)
result = pd.concat(dfs, axis=0)
result.to_csv('result/jp225_2021-2024.csv', index=False, header=False)


2021
2022
2023
2024


# Combine CSV files without name rules

In [7]:
import pandas as pd
dfs = []
files = ['result/AU_2010-2023.csv',
    'result/audusd_from_1-2024_to_11-2024.csv']

dfs=[]
for file in files:
    df = pd.read_csv(file, header=None)
    dfs.append(df)
    print(file)
result = pd.concat(dfs, axis=0)
result.to_csv('result/AU_2010-2024.csv', index=False, header=False)

result/AU_2010-2023.csv
result/audusd_from_1-2024_to_11-2024.csv


# Some processes

In [3]:
import pandas as pd

df = pd.read_csv('result/jp225_from_1-2021_to_12-2021.csv', header=None)
df[4] = df[4]*100
print(df.head(1))
df.to_csv('result/jp225_from_1-2021_to_12-2021.csv', index=False, header=False)

                     0        1            2            3     4      5
0  2021-01-04 02:00:00  27403.8  27798.81671  27217.98329  30.0  47.04


# Test env file

In [1]:
from dotenv import load_dotenv, dotenv_values
import os

load_dotenv()

server = os.getenv("SQL_SERVER")

HUNG-LAPTOP
